## Restaurant Graph Generator

In [39]:
parameters = {
    "features": {
        "attributes": True,
        "categories": (True,
                       [(1, 50)]),
        "city": True,
        "state": True,
        "temporal_skew": True,
        "temporal_alignment": False,
        "node_degree": False,
        "user_influence": True
    },
    "distance": 500, # Maximum distance between nodes
    "filename": "../graphs/restaurants_user_influence2",
}

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import json 

### Read Data

In [3]:
restaurants_df = pd.read_csv("../datasets/2017-2018_restaurants.csv")
edges_df = pd.read_csv("../datasets/2017-2018_restaurant-edges.csv")

In [4]:
restaurants_df.head(2)

,business_id,city,state,latitude,longitude,is_open,attributes,categories,hours,checkin_count,review_count,raw_stars,stars,weighted_stars,tip_count,first_date,last_date,visit_count,is_open_year_after
0,6iYb2HFDywm3zjuRg0shjw,Boulder,CO,40.017544,-105.283348,1,"{'RestaurantsTableService': 'True', 'WiFi': ""u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",79,49,3.714286,3.5,2.827977,4,2017-09-09 04:42:34,2021-01-22 05:20:38,132,True
1,tCbdrRPZA0oiIYSmHG3J0w,Portland,OR,45.588906,-122.593331,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",181,24,3.500000,3.5,2.734268,4,2010-03-09 16:02:04,2021-01-21 17:55:35,209,True


In [5]:
edges_df.head(3)

,id1,id2,distance
0,Q2vefh0tGhtCGQDK1FI7cw,ssK5vKQ_eN0VyGoYKOmkeQ,441
1,Q2vefh0tGhtCGQDK1FI7cw,tSZTPA7uERhWkKq_jbl3Eg,209
2,Q2vefh0tGhtCGQDK1FI7cw,bSy6VVJIdYPza1Bj9_Eicw,450


In [6]:
# Remove rows where stars is NaN
restaurants_df = restaurants_df[restaurants_df['stars'].notna()]

In [7]:
print(f"There are {len(restaurants_df)} restaurants and {len(edges_df)} edges.")

There are 29963 restaurants and 494203 edges.


In [8]:
visited_user_df = pd.read_csv("../datasets/2017-2018_visited_users.csv")
visited_user_df.head()

,business_id,user_ids
0,--6COJIAjkQwSUZci_4PJQ,"['kkSI0sYOzMXBzofb17U8Qw', 'mRyIfVHIJN6wwR3hnT..."
1,--UNNdnHRhsyFUbDgumdtQ,"['386nVS_BRsZBG5k3tO1LeQ', 'DEjRvKAJWCrjCaEP-F..."
2,--bbZa1KPYSmW0X4o3TUQw,"['6tJZrZYLmXLBx7HrpFmN-Q', 'tBRlSyCXalfAxLH2j9..."
3,--hkbIWgBKBOZq4VcNwdhQ,"['vO-RYeLAdBMkVOJu3fy7Kg', 'vUIydOj8gaNwXaTXTF..."
4,--oGYZhLMMvwfHmtyYJVxQ,"['vdMJFmbbDe0nDQr9-CjiGw', 'OMbCPxSkHKUldqFVwf..."


In [9]:
user_G = nx.read_gpickle("../graphs/2017-2018_user_network.gpickle")

### Calculate Popularity Category

In [10]:
from restaurant_utils import calculate_popularity

restaurants_df = calculate_popularity(restaurants_df)

### Extract attributes into columns
**Attributes supported:** RestaurantsPriceRange2, OutdoorSeating, RestaurantsGoodForGroups, BusinessAcceptsCreditCards, GoodForKids, RestaurantsDelivery, Caters, RestaurantsTakeOut, WheelchairAccessible, WiFi

In [11]:
from restaurant_utils import extract_attributes

if parameters["features"]["attributes"]:
    # Necessary as attributes are stored as JSON
    all_business_df = pd.read_json('../yelp/yelp_academic_dataset_business.json', lines=True)
    all_business_df = all_business_df[["business_id", "attributes"]]

    restaurants_df = extract_attributes(restaurants_df, all_business_df)

In [12]:
restaurants_df.head(3)

,business_id,city,state,latitude,longitude,is_open,categories,hours,checkin_count,review_count,...,RestaurantsPriceRange2,WiFi,OutdoorSeating,RestaurantsGoodForGroups,BusinessAcceptsCreditCards,GoodForKids,RestaurantsDelivery,Caters,WheelchairAccessible,RestaurantsTakeOut
0,6iYb2HFDywm3zjuRg0shjw,Boulder,CO,40.017544,-105.283348,1,"Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",79,49,...,2,1,1,1,1,0,1,1,1,1
1,tCbdrRPZA0oiIYSmHG3J0w,Portland,OR,45.588906,-122.593331,1,"Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",181,24,...,2,1,1,1,1,1,1,1,0,1
2,D4JtQNTI4X3KcbzacDJsMw,Vancouver,BC,49.251342,-123.101333,1,"Restaurants, Thai","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'...",42,28,...,2,0,1,1,0,1,1,1,0,1


### Get top categories pairs

In [13]:
from restaurant_utils import get_top_k_p_combinations, get_cat2idx, get_category_features
top_categories = []

for num_comb, topk in parameters["features"]["categories"][1]:
    top_categories += get_top_k_p_combinations(restaurants_df, num_comb, topk)

In [14]:
top_categories

[('Nightlife',),
 ('Bars',),
 ('American (Traditional)',),
 ('Coffee & Tea',),
 ('Sandwiches',),
 ('Breakfast & Brunch',),
 ('American (New)',),
 ('Pizza',),
 ('Fast Food',),
 ('Burgers',),
 ('Mexican',),
 ('Chinese',),
 ('Italian',),
 ('Specialty Food',),
 ('Seafood',),
 ('Event Planning & Services',),
 ('Salad',),
 ('Desserts',),
 ('Bakeries',),
 ('Japanese',),
 ('Cafes',),
 ('Grocery',),
 ('Beer',),
 ('Wine & Spirits',),
 ('Shopping',),
 ('Sushi Bars',),
 ('Caterers',),
 ('Ice Cream & Frozen Yogurt',),
 ('Chicken Wings',),
 ('Asian Fusion',),
 ('Food Trucks',),
 ('Cocktail Bars',),
 ('Pubs',),
 ('Delis',),
 ('Sports Bars',),
 ('Vegetarian',),
 ('Juice Bars & Smoothies',),
 ('Mediterranean',),
 ('Barbeque',),
 ('Thai',),
 ('Diners',),
 ('Steakhouses',),
 ('Arts & Entertainment',),
 ('Wine Bars',),
 ('Soup',),
 ('Tex-Mex',),
 ('Gluten-Free',),
 ('Vegan',),
 ('Vietnamese',),
 ('Convenience Stores',)]

In [15]:
cat2idx, idx2cat = get_cat2idx(restaurants_df, parameters["features"]["categories"][1][0][1])

In [16]:
restaurants_df = get_category_features(restaurants_df, cat2idx, top_categories)

In [17]:
restaurants_df.head(2)

,business_id,city,state,latitude,longitude,is_open,categories,hours,checkin_count,review_count,...,OutdoorSeating,RestaurantsGoodForGroups,BusinessAcceptsCreditCards,GoodForKids,RestaurantsDelivery,Caters,WheelchairAccessible,RestaurantsTakeOut,top_categories_vector,top_categories_combination_vector
0,6iYb2HFDywm3zjuRg0shjw,Boulder,CO,40.017544,-105.283348,1,"Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",79,49,...,1,1,1,0,1,1,1,1,"[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,tCbdrRPZA0oiIYSmHG3J0w,Portland,OR,45.588906,-122.593331,1,"Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",181,24,...,1,1,1,1,1,1,0,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


### Encode Extra: City and States

In [18]:
from restaurant_utils import get_encoder, encode_cities_states, to_categorical

In [19]:
encoder = get_encoder(restaurants_df)
restaurants_df = encode_cities_states(restaurants_df, encoder)

In [20]:
restaurants_df.head(3)

,business_id,city,state,latitude,longitude,is_open,categories,hours,checkin_count,review_count,...,BusinessAcceptsCreditCards,GoodForKids,RestaurantsDelivery,Caters,WheelchairAccessible,RestaurantsTakeOut,top_categories_vector,top_categories_combination_vector,city_idx,state_idx
0,6iYb2HFDywm3zjuRg0shjw,Boulder,CO,40.017544,-105.283348,1,"Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",79,49,...,1,0,1,1,1,1,"[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,1
1,tCbdrRPZA0oiIYSmHG3J0w,Portland,OR,45.588906,-122.593331,1,"Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",181,24,...,1,1,1,1,0,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",2,2
2,D4JtQNTI4X3KcbzacDJsMw,Vancouver,BC,49.251342,-123.101333,1,"Restaurants, Thai","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'...",42,28,...,0,1,1,1,0,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,3


### Temporal Popularity Skew

In [21]:
all_checkins_df = pd.read_json('../yelp/yelp_academic_dataset_checkin.json', lines=True)
all_checkins_df.head(3)

,business_id,date
0,--0r8K_AQ4FZfLsX3ZYRDA,2017-09-03 17:13:59
1,--0zrn43LEaB4jUWTQH_Bg,"2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010..."
2,--164t1nclzzmca7eDiJMw,"2010-02-26 02:06:53, 2010-02-27 08:00:09, 2010..."


In [22]:
from restaurant_utils import get_temporal_skew

if parameters["features"]["temporal_skew"]:
    restaurants_df = get_temporal_skew(restaurants_df, all_checkins_df)

In [23]:
restaurants_df.head(5)

,business_id,city,state,latitude,longitude,is_open,categories,hours,checkin_count,review_count,...,Caters,WheelchairAccessible,RestaurantsTakeOut,top_categories_vector,top_categories_combination_vector,city_idx,state_idx,date,temporal_profile,temporal_skew
0,6iYb2HFDywm3zjuRg0shjw,Boulder,CO,40.017544,-105.283348,1,"Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",79,49,...,1,1,1,"[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,1,"2017-09-10 04:48:12, 2017-09-10 04:49:28, 2017...","[0.14, 0.11, 0.11, 0.04, 0.02, 0.0, 0.01, 0.0,...",2.351
1,tCbdrRPZA0oiIYSmHG3J0w,Portland,OR,45.588906,-122.593331,1,"Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",181,24,...,1,0,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",2,2,"2010-04-22 05:31:33, 2010-05-09 18:24:50, 2010...","[0.04, 0.03, 0.03, 0.03, 0.01, 0.01, 0.0, 0.0,...",2.637
2,D4JtQNTI4X3KcbzacDJsMw,Vancouver,BC,49.251342,-123.101333,1,"Restaurants, Thai","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'...",42,28,...,1,0,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,3,"2010-11-06 02:53:03, 2010-11-29 02:16:55, 2010...","[0.1, 0.15, 0.22, 0.15, 0.05, 0.02, 0.0, 0.0, ...",2.210
3,ufCxltuh56FF4-ZFZ6cVhg,Orlando,FL,28.513265,-81.374707,1,"Restaurants, American (New), Bakeries, Dessert...","{'Tuesday': '11:0-18:0', 'Wednesday': '11:0-18...",42,38,...,1,0,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",4,4,"2012-08-29 22:10:36, 2012-09-11 18:11:11, 2012...","[0.0, 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0,...",2.219
4,dmbbf3AqeG61_QHRZi1M1w,Pine Castle,FL,28.450302,-81.380587,1,"Automotive, American (Traditional), Gas Statio...",NaN,4,3,...,0,0,1,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5,4,"2015-06-11 16:44:12, 2016-01-12 16:31:22, 2016...","[0.04, 0.08, 0.0, 0.0, 0.0, 0.0, 0.04, 0.0, 0....",2.466


In [24]:
if parameters["features"]["temporal_skew"]:
    print(restaurants_df["temporal_skew"].describe())

count    29963.000000
mean         2.316869
std          0.317786
min         -0.001000
25%          2.188000
50%          2.352000
75%          2.504000
max          3.166000
Name: temporal_skew, dtype: float64


### Generate Graph

In [25]:
G = nx.Graph()

In [26]:
from restaurant_utils import get_categories

# nodes
if parameters["features"]["attributes"]:
    features = restaurants_df[[
        "OutdoorSeating", "RestaurantsGoodForGroups", "BusinessAcceptsCreditCards", "WiFi", "RestaurantsPriceRange2",
        "GoodForKids", "RestaurantsDelivery", "Caters", "WheelchairAccessible", "RestaurantsTakeOut"
    ]].values
    print(f"Added {len(features[0])} features for attributes")

if parameters["features"]["categories"][0]:
    feat = np.stack(restaurants_df["top_categories_vector"].values)
    features = np.concatenate([features, feat], axis=1)

    if len(parameters["features"]["categories"][1]) > 1:  # category combinations exists
        feat = np.stack(restaurants_df["top_categories_combination_vector"].values)
        features = np.concatenate([features, feat], axis=1)
    
if parameters["features"]["city"]:
    feat = to_categorical(restaurants_df["city_idx"],
                          num_classes=restaurants_df["city_idx"].max() + 1)
    features = np.concatenate([features, feat], axis=1)
    print(f"Added {len(feat[0])} features for city")

if parameters["features"]["state"]:
    feat = to_categorical(restaurants_df["state_idx"],
                          num_classes=restaurants_df["state_idx"].max() + 1)
    features = np.concatenate([features, feat], axis=1)
    print(f"Added {len(feat[0])} features for state")

if parameters["features"]["temporal_skew"]:
    feat = restaurants_df["temporal_skew"].values
    count = len(features)
    feat = np.reshape(feat, (count, 1))
    features = np.concatenate([features, feat], axis=1)
    print(f"Added {len(feat[0])} features for temporal skew")
    
node_label = restaurants_df["popularity"].values  # [num_nodes,]

for index, row in restaurants_df.iterrows():
    G.add_node(row["business_id"],
               node_label=node_label[index],
               node_feature=features[index],
               node_type="restaurant")

Added 10 features for attributes
Added 365 features for city
Added 10 features for state
Added 1 features for temporal skew


In [27]:
# edges
for index, row in edges_df.iterrows():
    node1 = row["id1"]
    node2 = row["id2"]
    if node1 not in G.nodes or node2 not in G.nodes:
        continue
    dist = row["distance"]

    if dist <= parameters["distance"]:
        G.add_edge(node1, node2)

In [28]:
ids = list(G.nodes)

### Temporal Alignment of Restaurant to its Locality
Locality is defined as the restaurants it is connected to with an edge

In [29]:
if parameters["features"]["temporal_alignment"]:
    for index, row in restaurants_df.iterrows():
        node_id = row["business_id"]
        profile = row["temporal_profile"]

        neighbors = list(G[node_id])
        locality_profile = [0] * 24
        for neighbor_id in neighbors:
            neighbor_row = restaurants_df.loc[restaurants_df['business_id'] == neighbor_id]
            neighbor_profile = list(neighbor_row["temporal_profile"])[0]
            new_locality_profile = [locality_profile[i] + neighbor_profile[i] for i in range(24)]
            locality_profile = new_locality_profile

        if len(neighbors) > 0:
            locality_profile_normalized = [round(lp/len(neighbors), 2) for lp in locality_profile]
        else:
            locality_profile_normalized = locality_profile

        alignment = 0
        for i in range(24):
            alignment += (profile[i] - locality_profile_normalized[i])**2
        alignment = round(alignment, 3)

        G.nodes[node_id]["node_feature"] = np.append(G.nodes[node_id]["node_feature"], alignment)

        if index % 1000 == 0:
            print(index)

### Node Degree

In [30]:
if parameters["features"]["node_degree"]:
    degrees_dict = {node:val for (node, val) in G.degree()}
    max_degree = max([val for (node, val) in G.degree()])
    
    for node_id in ids:
        node_degree = round(degrees_dict[node_id] / max_degree, 3)
        G.nodes[node_id]["node_feature"] = np.append(G.nodes[node_id]["node_feature"], node_degree)

### User Influence

In [31]:
if parameters["features"]["user_influence"]:
    restaurants_visited_by_user = {}
    users_that_visited_restaurant = {}

    for index, row in visited_user_df.iterrows():
        restaurant_id = row["business_id"]
        user_ids = json.loads(row["user_ids"].replace("'", "\""))

        for user_id in user_ids:
            if user_id in restaurants_visited_by_user:
                restaurants_visited_by_user[user_id].append(restaurant_id)
            else:
                restaurants_visited_by_user[user_id] = [restaurant_id]
            
        users_that_visited_restaurant[restaurant_id] = user_ids 

In [32]:
if parameters["features"]["user_influence"]:
    user_edges = list(user_G.edges.data())
    friends = {}
    
    for edge in user_edges:
        node1 = edge[0]
        node2 = edge[1]
        
        if node1 in friends:
            friends[node1].append(node2)
        else:
            friends[node1] = [node2]
            
        if node2 in friends:
            friends[node2].append(node1)
        else:
            friends[node2] = [node1]

In [33]:
if parameters["features"]["user_influence"]:
    for node_id in ids:
        shared_friends = 0
        total = 1
        
        if node_id in users_that_visited_restaurant:
            users = users_that_visited_restaurant[node_id]
            
            for i in range(len(users)):
                for j in range(i + 1, len(users)):
                    total += 1
                    if users[i] in friends:
                        if users[j] in friends[users[i]]:
                            shared_friends += 1
        
        ratio = round(shared_friends / total, 3) 
        G.nodes[node_id]["node_feature"] = np.append(G.nodes[node_id]["node_feature"], shared_friends)
        G.nodes[node_id]["node_feature"] = np.append(G.nodes[node_id]["node_feature"], ratio)

### Summary

In [34]:
for node_id in ids:
    G.nodes[node_id]["node_feature"] = torch.FloatTensor(G.nodes[node_id]["node_feature"])

In [35]:
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"Number of features: {len(list(G.nodes.data())[0][1]['node_feature'])}")

Number of nodes: 29963
Number of edges: 491371
Number of features: 439


In [36]:
list(G.nodes.data())[10][1]['node_feature']

tensor([ 0.0000,  0.0000,  1.0000,  0.0000,  2.0000,  0.0000,  0.0000,  1.0000,
         0.0000,  1.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

### Write graph to disk

In [37]:
nx.write_gpickle(G, parameters["filename"] + ".gpickle")

In [38]:
with open(parameters["filename"] + ".json", "w") as outfile: 
    json.dump(parameters, outfile)

In [42]:
restaurants_df[["business_id", "visit_count"]]

,business_id,visit_count
0,6iYb2HFDywm3zjuRg0shjw,132
1,tCbdrRPZA0oiIYSmHG3J0w,209
2,D4JtQNTI4X3KcbzacDJsMw,73
3,ufCxltuh56FF4-ZFZ6cVhg,85
4,dmbbf3AqeG61_QHRZi1M1w,8
...,...,...
29958,yQL8SrSETbbCI1U5esVJQw,604
29959,r5Uag1JqYjr2nbxQCVqm8A,781
29960,Q78fYV6B6P6GmX07YVgi4g,217
29961,uXdQkuEtvLAzfc3MsO-sTQ,168


In [44]:
G.nodes()["tCbdrRPZA0oiIYSmHG3J0w"]

{'node_label': 2,
 'node_feature': tensor([1.0000, 1.0000, 1.0000, 1.0000, 2.0000, 1.0000, 1.0000, 1.0000, 0.0000,
         1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.